In [1]:
import boto3
import botocore
import pandas as pd
from IPython.display import display, Markdown

from io import StringIO
import os

In [2]:
import findspark
findspark.init()
from pyspark import SparkContext
sc=SparkContext()
sc

<SparkContext master=yarn appName=pyspark-shell>

In [3]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.getOrCreate()
spark

In [12]:
data1970 = spark.read.option("header", "false").csv("s3://noaa-global-hourly-pds/1970/*.csv")

In [13]:
data1970.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)
 |-- _c12: string (nullable = true)
 |-- _c13: string (nullable = true)
 |-- _c14: string (nullable = true)
 |-- _c15: string (nullable = true)
 |-- _c16: string (nullable = true)
 |-- _c17: string (nullable = true)
 |-- _c18: string (nullable = true)
 |-- _c19: string (nullable = true)
 |-- _c20: string (nullable = true)
 |-- _c21: string (nullable = true)
 |-- _c22: string (nullable = true)
 |-- _c23: string (nullable = true)
 |-- _c24: string (nullable = true)
 |-- _c25: string (nullable = true)
 |-- _c26: string (nullable = true)
 |-- _c27: string (nullable = tru

In [20]:
data1970.count()

8694985

In [14]:
data1970.cache()

DataFrame[_c0: string, _c1: string, _c2: string, _c3: string, _c4: string, _c5: string, _c6: string, _c7: string, _c8: string, _c9: string, _c10: string, _c11: string, _c12: string, _c13: string, _c14: string, _c15: string, _c16: string, _c17: string, _c18: string, _c19: string, _c20: string, _c21: string, _c22: string, _c23: string, _c24: string, _c25: string, _c26: string, _c27: string, _c28: string, _c29: string, _c30: string, _c31: string]

In [15]:
data1970.where('_c0=="STATION"').count()

3006

In [18]:
data19702=data1970.where('_c0!= "STATION" ')

In [19]:
data19702.count()

8691979

In [59]:
data1970.show()

+-----------+-------------------+------+--------+---------+---------+--------------------+-----------+---------+---------------+--------------+-----------+------------+-------+-------+-------+-----------+------------------+------------------+------------------+------------------+-----------------+-----------------+-----------------+-----------------+--------------------+---------------+----+----+----+----+----------------+
|        _c0|                _c1|   _c2|     _c3|      _c4|      _c5|                 _c6|        _c7|      _c8|            _c9|          _c10|       _c11|        _c12|   _c13|   _c14|   _c15|       _c16|              _c17|              _c18|              _c19|              _c20|             _c21|             _c22|             _c23|             _c24|                _c25|           _c26|_c27|_c28|_c29|_c30|            _c31|
+-----------+-------------------+------+--------+---------+---------+--------------------+-----------+---------+---------------+--------------+---

In [21]:
#add index row
from pyspark.sql import SparkSession
#from pyspark.sql import functions._
#import pyspark.spark.sql.{SQLContext, Row, DataFrame, Column}
from pyspark.sql import Row,DataFrame,Column
from pyspark.sql import SQLContext
#from pyspark.ml import feature.VectorAssembler

In [77]:
import pyspark.sql.functions as f
from pyspark.sql.functions import concat, col, lit

data2_converted = data19702.select(col("_c0"), col("_c12"), col("_c13"), col("_c16"))
#data2_converted.show()
data2_converted2 = data2_converted.select(col("_c0").alias("station"), f.split(data2_converted['_c12'], ',')[0].alias("vis"),f.split(data2_converted['_c13'], ',')[0].alias("temp"), f.split(data2_converted['_c16'], ',')[1].alias("liquid"),f.split(data2_converted['_c16'], ',')[0].alias("liqhr"))
data2_converted2.show()

+-----------+------+-----+------+-----+
|    station|   vis| temp|liquid|liqhr|
+-----------+------+-----+------+-----+
|72572024127|012800|-0011|  0000|   01|
|72572024127|004800|-0011|  0000|   01|
|72572024127|002400|-0022|  0003|   01|
|72572024127|004800|-0022|  0000|   01|
|72572024127|001600|-0022|  0005|   01|
|72572024127|001600|-0028|  0005|   01|
|72572024127|006400|-0033|  0000|   01|
|72572024127|006400|-0028|  0000|   01|
|72572024127|000000|-0039|  0000|   01|
|72572024127|000000|-0083|  0000|   01|
|72572024127|032000|-0106|  0000|   01|
|72572024127|000000|-0106|  0000|   01|
|72572024127|000000|-0122|  0000|   01|
|72572024127|048000|-0128|  0000|   01|
|72572024127|048000|-0133|  0000|   01|
|72572024127|048000|-0128|  0000|   01|
|72572024127|048000|-0122|  0000|   01|
|72572024127|048000|-0128|  0000|   01|
|72572024127|032000|-0094|  0000|   01|
|72572024127|032000|-0061|  0000|   01|
+-----------+------+-----+------+-----+
only showing top 20 rows



In [79]:
data2_converted3=data2_converted2.withColumn("temp", data2_converted2["temp"].cast("float"))
data2_converted3=data2_converted3.withColumn("vis", data2_converted3["vis"].cast("float"))
data2_converted3=data2_converted3.withColumn("liquid", data2_converted3["liquid"].cast("float"))
data2_converted3=data2_converted3.withColumn("liqhr", data2_converted3["liqhr"].cast("float"))

In [62]:
data2_converted3.sort('temp', ascending=True).show()

+-----------+-------+------+
|    station|    vis|  temp|
+-----------+-------+------+
|89009090001|11200.0|-750.0|
|89009090001|11200.0|-745.0|
|89009090001|11200.0|-739.0|
|89009090001|11200.0|-734.0|
|89009090001|11200.0|-734.0|
|89009090001|11200.0|-734.0|
|89009090001|11200.0|-728.0|
|89009090001|11200.0|-728.0|
|89009090001|11200.0|-722.0|
|89009090001|11200.0|-722.0|
|89009090001|11200.0|-722.0|
|89009090001|11200.0|-722.0|
|89009090001|11200.0|-722.0|
|89009090001|11200.0|-717.0|
|89009090001|11200.0|-717.0|
|89009090001| 8000.0|-717.0|
|89009090001| 8000.0|-711.0|
|89009090001|11200.0|-711.0|
|89009090001|11200.0|-711.0|
|89009090001|11200.0|-711.0|
+-----------+-------+------+
only showing top 20 rows



In [81]:
# Rescale since the original data is scaled by 10
data2_converted4=data2_converted3.withColumn("temp", data2_converted3["temp"]/10)
data2_converted4=data2_converted3.withColumn("liquid", data2_converted3["liquid"]/10/data2_converted3["liqhr"])
data2_converted4.show()

+-----------+-------+------+------+-----+
|    station|    vis|  temp|liquid|liqhr|
+-----------+-------+------+------+-----+
|72572024127|12800.0| -11.0|   0.0|  1.0|
|72572024127| 4800.0| -11.0|   0.0|  1.0|
|72572024127| 2400.0| -22.0|   0.3|  1.0|
|72572024127| 4800.0| -22.0|   0.0|  1.0|
|72572024127| 1600.0| -22.0|   0.5|  1.0|
|72572024127| 1600.0| -28.0|   0.5|  1.0|
|72572024127| 6400.0| -33.0|   0.0|  1.0|
|72572024127| 6400.0| -28.0|   0.0|  1.0|
|72572024127|    0.0| -39.0|   0.0|  1.0|
|72572024127|    0.0| -83.0|   0.0|  1.0|
|72572024127|32000.0|-106.0|   0.0|  1.0|
|72572024127|    0.0|-106.0|   0.0|  1.0|
|72572024127|    0.0|-122.0|   0.0|  1.0|
|72572024127|48000.0|-128.0|   0.0|  1.0|
|72572024127|48000.0|-133.0|   0.0|  1.0|
|72572024127|48000.0|-128.0|   0.0|  1.0|
|72572024127|48000.0|-122.0|   0.0|  1.0|
|72572024127|48000.0|-128.0|   0.0|  1.0|
|72572024127|32000.0| -94.0|   0.0|  1.0|
|72572024127|32000.0| -61.0|   0.0|  1.0|
+-----------+-------+------+------

In [64]:
# Drop rows with NA or missing values in pyspark
data2_converted4_dropna=data2_converted4.dropna()

In [65]:
data2_converted4_dropna.sort('temp', ascending=False).show()

+-----------+-------+-----+
|    station|    vis| temp|
+-----------+-------+-----+
|99999903102|11200.0|999.9|
|99999903102|19200.0|999.9|
|99999903102| 4800.0|999.9|
|99999903102| 4800.0|999.9|
|99999903102|12800.0|999.9|
|99999903102|16000.0|999.9|
|99999903102| 8000.0|999.9|
|99999903102|11200.0|999.9|
|99999903102|14400.0|999.9|
|99999903102| 6400.0|999.9|
|99999903102|24000.0|999.9|
|99999903102| 6400.0|999.9|
|99999903102|  500.0|999.9|
|99999903102| 9600.0|999.9|
|99999903102| 4800.0|999.9|
|99999903102|12800.0|999.9|
|99999903102| 4800.0|999.9|
|99999903102| 8000.0|999.9|
|99999903102| 4800.0|999.9|
|99999903102| 8000.0|999.9|
+-----------+-------+-----+
only showing top 20 rows



In [68]:
#drop 9999 as NA
data2_converted4_dropna = data2_converted4_dropna.filter(data2_converted4_dropna.temp != '999.9')
data2_converted4_dropna = data2_converted4_dropna.filter(data2_converted4_dropna.vis != '999999')
data2_converted4_dropna.sort('temp', ascending=False).show()


+-----------+-------+----+
|    station|    vis|temp|
+-----------+-------+----+
|99999923158|64000.0|48.9|
|99999923158|64000.0|48.9|
|99999923158|64000.0|48.9|
|99999923158|64000.0|48.3|
|74724023168|32000.0|47.9|
|99999903148|32000.0|47.8|
|99999923158|64000.0|47.8|
|74724023168|40000.0|47.4|
|72278623104|48000.0|47.4|
|72386523112|56000.0|47.4|
|99999923158|64000.0|47.2|
|99999903148|40000.0|47.2|
|72278623104|48000.0|46.8|
|72278623104|48000.0|46.8|
|72386523112|40000.0|46.8|
|74724023168|32000.0|46.8|
|74724023168|40000.0|46.8|
|99999903148|40000.0|46.7|
|99999903148|32000.0|46.7|
|74724023168|48000.0|46.3|
+-----------+-------+----+
only showing top 20 rows



In [69]:
data2_converted4_dropna.sort('vis', ascending=False).show()

+-----------+--------+----+
|    station|     vis|temp|
+-----------+--------+----+
|78016013601|160000.0|19.4|
|78016013601|160000.0|17.8|
|78016013601|160000.0|18.9|
|78016013601|160000.0|19.4|
|78016013601|160000.0|18.9|
|78016013601|160000.0|19.4|
|78016013601|160000.0|20.0|
|78016013601|160000.0|19.4|
|78016013601|160000.0|20.0|
|78016013601|160000.0|18.9|
|78016013601|160000.0|19.4|
|78016013601|160000.0|18.9|
|78016013601|160000.0|19.4|
|78016013601|160000.0|18.9|
|78016013601|160000.0|19.4|
|78016013601|160000.0|18.9|
|78016013601|160000.0|20.0|
|78016013601|160000.0|20.0|
|78016013601|160000.0|20.0|
|78016013601|160000.0|20.0|
+-----------+--------+----+
only showing top 20 rows



In [70]:
data2_converted4_dropna.count()

8182534

In [30]:
data2_converted4_dropna.agg({"temp": "avg"}).show()

+-----------------+
|        avg(temp)|
+-----------------+
|7.906684285046325|
+-----------------+



In [33]:
data2_converted4_dropna.agg({"temp": "std"}).show()

+------------------+
|      stddev(temp)|
+------------------+
|15.781198986770178|
+------------------+



In [71]:
data2_converted4_dropna.agg({"vis": "avg"}).show()

+------------------+
|          avg(vis)|
+------------------+
|22431.092017216182|
+------------------+



In [72]:
data2_converted4_dropna.agg({"vis": "std"}).show()

+------------------+
|       stddev(vis)|
+------------------+
|18808.127321879572|
+------------------+



In [31]:
data1970mean = data2_converted4_dropna.agg({"temp": "avg"})

In [49]:
m1970temp = data1970mean.collect()[0]["avg(temp)"] 

In [34]:
data1970std = data2_converted4_dropna.agg({"temp": "std"})

In [55]:
s1970temp = data1970std.collect()[0]["stddev(temp)"] 

In [56]:
stats = spark.createDataFrame(
    [
        (m1970, s1970, 1970), # create your data here, be consistent in the types.
        #(2, 'bar'),
    ],
    ['mean', 'std', 'year'] # add your columns label here
)

In [57]:
stats.show()

+-----------------+------------------+----+
|             mean|               std|year|
+-----------------+------------------+----+
|7.906684285046328|15.781198986770175|1970|
+-----------------+------------------+----+



In [68]:
s3 = boto3.resource('s3')
s3

s3.ServiceResource()

In [69]:
bucket = s3.Bucket(name="noaa-global-hourly-pds")
bucket

s3.Bucket(name='open-data-noaa')

In [70]:
#Get File Name List
prefix = "1970/"
#prefix = "2010/"
#prefix = "2005/"
#prefix = "2000/"
#prefix = "1995/"
#prefix = "1990/"
#prefix = "1985/"
#prefix = "1980/"
#prefix = "1975/"
#prefix = "1970/"
FilesNotFound = True
FileNameList=[]
for obj in bucket.objects.filter(Prefix=prefix):
    FileNameList.append('{0}{1}/{2}'.format('s3://',bucket.name, obj.key))
    #print('{0}/{1}'.format(bucket.name, obj.key))
    FilesNotFound = False
if FilesNotFound:
     print("ALERT", "No file in {0}/{1}".format(bucket, prefix))


In [ ]:
FileNameList

In [104]:
#import random
#desired_pct = 1.65
#random.seed(20)
#file_sample = random.sample(FileNameList, int(len(FileNameList) * desired_pct / 100))

In [77]:
len(FileNameList)

3006

In [49]:
import random
desired_pct = 20
random.seed(20)
file_sample = random.sample(FileNameList, int(len(FileNameList) * desired_pct / 100))

In [50]:
len(file_sample)

601

## Merge Data

In [80]:
from py4j.protocol import Py4JJavaError

def file_exist(sc, file):
    try:
        rdd = sc.textFile(file)
        rdd.take(1)
        return True
    except Py4JJavaError as e:
        return False

In [83]:
#Merge
rdd = sc.textFile(FileNameList[0])
i=0
rdd_all=rdd
rdd.count()

2498

In [ ]:
rdd1 = sc.parallelize([1, 2, 3])
rdd2 = sc.parallelize([4, 5, 6])
rdd3 = sc.parallelize([7, 8, 9])

rdd = sc.union([rdd1, rdd2, rdd3])

In [ ]:
RDDs = sc.parallelize([["panda", 0], ["pink", 3]])
RDDList = []
for i in range(0,len(RDDs.first())):    
    RDDList.append(RDDs.map(lambda x: [x[i]]).collect())

print (RDDList[0])
print (RDDList[1])

In [84]:
while i < len(FileNameList)/7:
    if file_exist(sc, FileNameList[i]):
        #print(FileNameList[i])
        rdd_sep=sc.textFile(FileNameList[i])
        rdd_all=rdd_all.union(rdd_sep)
        
    else:
        print('File does not exist, skipping: ' + FileNameList[i])
    i += 1
rdd_all.count()

1193453

In [ ]:
rdd_all.count()

In [ ]:
  
rdd_full=sc.union([rdd_all, rdd_all2])    
rdd_all.count()

In [27]:
#Save
rdd_all.repartition(1).saveAsTextFile("s3://open-data-noaa/1970sample_v2")

In [28]:
df=spark.read.format("csv").option("header", "true").load("s3://open-data-noaa/1970sample_v2")

In [29]:
df.head(5)

[Row(STATION='99999993729', DATE='1970-01-01T00:00:00', SOURCE='E', LATITUDE='35.2326', LONGITUDE='-75.6219', ELEVATION='3.4', NAME='CAPE HATTERAS BILLY MITCHELL FIELD, NC US', REPORT_TYPE='SAO  ', CALL_SIGN='HAT  ', QUALITY_CONTROL='V020', WND='290,5,N,0057,5', CIG='22000,5,9,N', VIS='016000,5,9,9', TMP='+0128,4', DEW='+0061,5', SLP='10139,4', AA1='01,0000,9,5', GA1='99,9,+99999,9,00,4', GA2='99,9,+99999,9,99,9', GA3='99,9,+99999,9,99,9', GA4='99,9,+99999,9,99,9', GD1='4,08,5,+99999,9,9', GD2='0,00,5,+99999,9,9', GD3='0,00,5,+99999,9,9', GD4='0,00,5,+99999,9,9', GF1='08,06,5,99,9,99,9,99999,9,99,9,99,9', MA1='99999,9,10135,4', MW1=None, MW2=None, MW3=None, EQD='N01 00000JPWTH 1'),
 Row(STATION='99999993729', DATE='1970-01-01T03:00:00', SOURCE='E', LATITUDE='35.2326', LONGITUDE='-75.6219', ELEVATION='3.4', NAME='CAPE HATTERAS BILLY MITCHELL FIELD, NC US', REPORT_TYPE='SAO  ', CALL_SIGN='HAT  ', QUALITY_CONTROL='V020', WND='060,5,N,0077,5', CIG='22000,5,9,N', VIS='016000,5,9,9', TMP='+0

In [30]:
df.write.csv("s3://open-data-noaa/1970samples_v2", sep=',', header=True,mode="overwrite")

In [82]:
sc.stop()